In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data_files = {
  "Family":              "familyxx.csv",
  "Household":           "househld.csv",
  "Injury Episode":      "injpoiep.csv",
  "Person":              "personsx.csv",
  "Sample Child":        "samchild.csv",
  "Sample Adult":        "samadult.csv",
  "Sample Adult Cancer": "cancerxx.csv"
}

In [3]:
# war history, economic status?

In [4]:
# diet, cancer?